In [100]:
#import package
import pandas as pd
from geopy.distance import geodesic

pd.set_option('display.max_columns', None)

In [101]:
#import data
merged = pd.read_csv('/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/merged.csv')
metro = pd.read_csv('/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/Metro_Stations_in_DC.csv')

In [102]:
merged = merged.drop(columns=['Unnamed: 0'])
merged

,SHIFT,OFFENSE,WARD,LATITUDE,LONGITUDE,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ward_1,ward_2,ward_3,ward_4,ward_5,ward_6,ward_7,ward_8,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT
0,MIDNIGHT,THEFT F/AUTO,5.0,38.918707,-76.962533,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,MIDNIGHT,ASSAULT W/DANGEROUS WEAPON,6.0,38.907643,-77.009053,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,MIDNIGHT,THEFT F/AUTO,6.0,38.901323,-77.016914,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,DAY,THEFT F/AUTO,6.0,38.883239,-77.018730,2017-10-27,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0
4,DAY,THEFT/OTHER,5.0,38.920933,-76.997157,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147518,DAY,BURGLARY,5.0,38.913022,-76.975955,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
147519,DAY,THEFT F/AUTO,7.0,38.874610,-76.967094,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0
147520,DAY,THEFT F/AUTO,5.0,38.951338,-76.991691,2021-02-03,37365.0,166.0,6.0,106.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
147521,DAY,ASSAULT W/DANGEROUS WEAPON,8.0,38.852403,-76.982313,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [103]:
#creating new column for tuple of x and y coordinates
merged['coords'] = tuple(zip(merged.LONGITUDE, merged.LATITUDE))
metro['coords2'] = tuple(zip(metro.X, metro.Y))

In [104]:
#distance from metro
distance = []
for index, row in merged.iterrows():
    for index2, row2 in metro.iterrows():
        x = geodesic(row['coords'], row2['coords2']).miles
        z = [index, index2, x]
        distance.append(z)
df = pd.DataFrame(distance, columns = ['index', 'metro_stop_id', 'distance'])
df2 = df.groupby(['index'], as_index=False).agg(
{
    'distance':'min'
})
df2

,index,distance
0,0,1.127080
1,1,0.191281
2,2,0.083638
3,3,0.135380
4,4,0.084503
...,...,...
147518,147518,0.418831
147519,147519,0.718883
147520,147520,0.345018
147521,147521,0.442869


In [109]:
#join metro distance into crime data
df3 = pd.concat([merged, df2], axis=1)
df3

,SHIFT,OFFENSE,WARD,LATITUDE,LONGITUDE,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ward_1,ward_2,ward_3,ward_4,ward_5,ward_6,ward_7,ward_8,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,coords,index,distance
0,MIDNIGHT,THEFT F/AUTO,5.0,38.918707,-76.962533,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,"(-76.96253311529999, 38.9187067412)",0,1.127080
1,MIDNIGHT,ASSAULT W/DANGEROUS WEAPON,6.0,38.907643,-77.009053,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,"(-77.0090530223, 38.907642772399996)",1,0.191281
2,MIDNIGHT,THEFT F/AUTO,6.0,38.901323,-77.016914,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,"(-77.0169144784, 38.901323441100004)",2,0.083638
3,DAY,THEFT F/AUTO,6.0,38.883239,-77.018730,2017-10-27,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,"(-77.0187301806, 38.8832386792)",3,0.135380
4,DAY,THEFT/OTHER,5.0,38.920933,-76.997157,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,"(-76.99715700510001, 38.9209329952)",4,0.084503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147518,DAY,BURGLARY,5.0,38.913022,-76.975955,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,"(-76.9759548322, 38.9130221422)",147518,0.418831
147519,DAY,THEFT F/AUTO,7.0,38.874610,-76.967094,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,"(-76.9670938752, 38.874610442)",147519,0.718883
147520,DAY,THEFT F/AUTO,5.0,38.951338,-76.991691,2021-02-03,37365.0,166.0,6.0,106.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,"(-76.99169130060001, 38.9513383536)",147520,0.345018
147521,DAY,ASSAULT W/DANGEROUS WEAPON,8.0,38.852403,-76.982313,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,"(-76.98231291260001, 38.852403147800004)",147521,0.442869


In [110]:
#dropping columns
#del df3['Unnamed: 0']
del df3['coords']
del df3['index']
#del df3['XBLOCK']
#del df3['YBLOCK']
#del df3['DISTRICT']
#del df3['METHOD']

df3

,SHIFT,OFFENSE,WARD,LATITUDE,LONGITUDE,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ward_1,ward_2,ward_3,ward_4,ward_5,ward_6,ward_7,ward_8,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,distance
0,MIDNIGHT,THEFT F/AUTO,5.0,38.918707,-76.962533,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1.127080
1,MIDNIGHT,ASSAULT W/DANGEROUS WEAPON,6.0,38.907643,-77.009053,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.191281
2,MIDNIGHT,THEFT F/AUTO,6.0,38.901323,-77.016914,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.083638
3,DAY,THEFT F/AUTO,6.0,38.883239,-77.018730,2017-10-27,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.135380
4,DAY,THEFT/OTHER,5.0,38.920933,-76.997157,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.084503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147518,DAY,BURGLARY,5.0,38.913022,-76.975955,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.418831
147519,DAY,THEFT F/AUTO,7.0,38.874610,-76.967094,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0.718883
147520,DAY,THEFT F/AUTO,5.0,38.951338,-76.991691,2021-02-03,37365.0,166.0,6.0,106.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.345018
147521,DAY,ASSAULT W/DANGEROUS WEAPON,8.0,38.852403,-76.982313,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.442869


In [113]:
df3.dtypes

SHIFT                             object
OFFENSE                           object
WARD                             float64
LATITUDE                         float64
LONGITUDE                        float64
date                              object
total_cases                      float64
daily_cases                      float64
daily_death                      float64
OUT_POSITIVE_QUARANTINE_PSMPD    float64
MO                                 int64
TU                                 int64
WE                                 int64
TH                                 int64
FR                                 int64
SA                                 int64
SU                                 int64
JAN                                int64
FEB                                int64
MAR                                int64
APR                                int64
MAY                                int64
JUN                                int64
JUL                                int64
AUG             

In [115]:
#split data into covid not covid
covid_start_date = pd.to_datetime("2020-03-11").date()
df3['date'] = pd.to_datetime(df3['date']).dt.date

pre_covid_time = df3[(df3['date'] <= covid_start_date)]
covid_time = df3[(df3['date'] > covid_start_date)] 

In [119]:
print(pre_covid_time.info())
print(covid_time.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104359 entries, 0 to 146071
Data columns (total 50 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   SHIFT                          104359 non-null  object 
 1   OFFENSE                        104359 non-null  object 
 2   WARD                           104359 non-null  float64
 3   LATITUDE                       104359 non-null  float64
 4   LONGITUDE                      104359 non-null  float64
 5   date                           104359 non-null  object 
 6   total_cases                    104359 non-null  float64
 7   daily_cases                    104359 non-null  float64
 8   daily_death                    104359 non-null  float64
 9   OUT_POSITIVE_QUARANTINE_PSMPD  104359 non-null  float64
 10  MO                             104359 non-null  int64  
 11  TU                             104359 non-null  int64  
 12  WE                            

In [120]:
#export to DF
df3.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/crime_covid_metro_11_28_2021.csv')
pre_covid_time.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/pre_covid_time_11_28_2021.csv')
covid_time.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/covid_time_11_28_2021.csv')
